In [ ]:
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

train_data_path = "./Classification/train/"
val_data_path = "./Classification/val/"
test_data_path = "./Classification/test/"

transform_pipeline = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform_pipeline)
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=transform_pipeline)
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=transform_pipeline)

batch_size = 64
train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

class CNNNet(nn.Module):
    def __init__(self, num_classes=2):
        super(CNNNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNNet(num_classes=len(train_data.classes)).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device='cpu'):
    model.to(device)
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss_value = loss_fn(outputs, targets)
            loss_value.backward()
            optimizer.step()
            training_loss += loss_value.item()
        training_loss /= len(train_loader)

        model.eval()
        num_correct = 0
        num_examples = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss_value = loss_fn(outputs, targets)
                valid_loss += loss_value.item()
                preds = torch.argmax(F.softmax(outputs, dim=1), dim=1)
                num_correct += (preds == targets).sum().item()
                num_examples += targets.size(0)
        valid_loss /= len(val_loader)
        accuracy = (num_correct / num_examples) * 100

        print(f'Epoch: {epoch + 1}, Training Loss: {training_loss:.2f}, Validation Loss: {valid_loss:.2f}, Accuracy: {accuracy:.2f}%')

train(model, optimizer, nn.CrossEntropyLoss(), train_data_loader, val_data_loader, epochs=20, device=device)
